<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Carga de librerías</span>

In [1]:
import ultralytics
import sklearn
from ultralytics import YOLO
from ultralytics import settings
import os
import torch
import subprocess
import gdown
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import shutil
from sklearn.model_selection import train_test_split
import yaml
from threading import Thread
from queue import Queue
from PIL import Image
import re
from pathlib import Path
import cv2
import tempfile

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Instalación de herramientas de etiquetado</span>

In [12]:
# Función para instalar los paquetes

def package_verification(package):
    try:
        output = os.popen('pip list').read()
        return f'{package}' in output
    except Exception as e:
        print(f'Error en la verificación: {e}')
        return False

def package_installation(package_name):
    if package_verification(package_name):
        print(f"El paquete '{package_name}' ya está instalado.")
    else:
        print('Instalando el paquete... ')
        try:
            os.system(f'pip install {package_name}')
            print(f'libreria {package_name} instalada correctamente! ')
        except Exception as e:
            print(f'Error en la instalación de {package_name}: {e}')

In [22]:
package_installation('labelme') # Instalación de la herramienta de etiquetado Labelme

El paquete 'labelme' ya está instalado.


In [ ]:
# Abrir la herramienta para el proceso de etiquetado 
try:
    subprocess.check_call(["labelme"]) 
except Exception as e:
    print(f'No fue posible abrir labelme: {e}')

In [16]:
package_installation('labelme2yolo') # Instalación de la herramienta de conversión para el formato JSON de labelme a formato texto requerido por los modelos de detección de objetos YOLO.

El paquete 'labelme2yolo' ya está instalado.


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Carga rutas</span>

In [2]:
current_folder= os.getcwd() # Obtener ubicación actual
main_folder= os.path.abspath(os.path.join(current_folder, "../../../")) # Carpeta main del proyecto
main_staticos= os.path.abspath(os.path.join(main_folder))+'\\dist' # Carpeta main de los archivos estaticos
main_production= os.path.abspath(main_folder) +'\\src' # Carpeta main de los archivos listos para producción 
dir_prueba= os.path.abspath(main_folder) + "\\src\\assets_2"  # Carpeta de imagenes prueba para el proceso de predecir

training_structure="lote2_2" # Indica el nombre de la carpeta que tiene los archivos para el entrenamiento (image, json)
dir_train= os.path.abspath(main_staticos) + f'\\{training_structure}'

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Configuración de rutas de entrenamiento</span>

In [5]:
# Función para separar la carpeta global de todos los datos en un 80% entrenamiento y 20% validación. Utilice esta función solo si no ha ordenado los datos en una carpeta para entrenamiento y otra para validación.

def file_separation(orig_folder):
    file_list = os.listdir(orig_folder)
    train, val= train_test_split(file_list, test_size=0.2, random_state=100) # Se divide el set de datos entrenamiento y validación aleatoriamente con una misma semilla
    if not os.path.exists("train"):
        train_path = os.path.join(orig_folder, "train")
        os.makedirs(train_path, exist_ok=True)
    move_files(train, orig_folder, train_path)
    
    if not os.path.exists("val"):
        val_path = os.path.join(orig_folder, "val")
        os.makedirs(val_path, exist_ok=True)
    move_files(val, orig_folder, val_path)

def move_files(file_list, orig_folder, dest_folder):
    for file in file_list:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            orig_image_path = os.path.join(orig_folder, file)
            json_name = os.path.splitext(file)[0] + ".json"
            
            orig_json_path = os.path.join(orig_folder, json_name)
            dest_image_path = os.path.join(dest_folder, file)
            dest_json_path = os.path.join(dest_folder, json_name)
            
            shutil.copy(orig_image_path, dest_image_path)
            shutil.copy(orig_json_path, dest_json_path)

file_separation(dir_train)

Cantidad de archivos de entrenamiento: 399
Cantidad de archivos de validación: 98


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Conversión a formato YOLO</span>

In [3]:
# Función para convertir la carpeta de entrenamiento y validación en formato YOLO
# Hacer que permita realizar la conversion si ya se tiene separado
def convert_to_yolo_format(training_directory, validation_directory):
    try:
        subprocess.check_call(["labelme2yolo", "--json_dir", training_directory])
        print(f'Conversión completada para la carpeta {training_directory}')
        subprocess.check_call(["labelme2yolo", "--json_dir", validation_directory])
        print(f'Conversión completada para la carpeta {validation_directory}')
    except subprocess.CalledProcessError as e:
        print(f'Error al ejecutar función labelme2yolo: {e}')

train_path= os.path.abspath(dir_train) + "\\train"
val_path= os.path.abspath(dir_train) + "\\val"

convert_to_yolo_format(train_path, val_path)

Conversión completada para la carpeta c:\Users\matrix\pruebayolo\proyecto_yolo\dist\lala\train
Conversión completada para la carpeta c:\Users\matrix\pruebayolo\proyecto_yolo\dist\lala\val


In [ ]:
# Organización de la estructura del repositorio YOLODataset (resultado de la conversión .json a formato YOLO) creado por defecto

def reorganization(path):
    try:
        new_path = os.path.abspath(os.path.join(path, "YOLODataset"))
        main_folders = os.listdir(new_path)
        for folder_instance in main_folders:
            secondary_folder_path = os.path.join(new_path, folder_instance)
            if os.path.isdir(secondary_folder_path):
                secondary_folders = os.listdir(secondary_folder_path)
                for secondary_folder in secondary_folders:
                    subfolder_secondary_path = os.path.join(secondary_folder_path, secondary_folder)
                    try:
                        if os.listdir(subfolder_secondary_path) != []:
                            for file in os.listdir(subfolder_secondary_path):
                                orig_path = os.path.join(subfolder_secondary_path, file)
                                dest_path = os.path.abspath(os.path.join(subfolder_secondary_path, "../"))
                                new_dest_path = os.path.join(dest_path, file)
                                shutil.move(orig_path, new_dest_path)
                            os.rmdir(subfolder_secondary_path)
                        else:
                            os.rmdir(subfolder_secondary_path)
                    except NotADirectoryError as e:
                        print(f'Error: the element is not a folder: {e}')
        return new_path
    except FileNotFoundError as e:
        print(f'Error: {e}')

yolo_train_data= reorganization(train_path)
yolo_val_data= reorganization(val_path)    

In [ ]:
# Función para mover las carpetas yolo 

def acomodar_rutas_dataset(main_staticos, dataset_train, dataset_val):
    try:
        if not os.path.exists("data_afectacion"):
            try: 
                paths_segmentation= os.path.join(main_staticos, "data_afectacion")
                os.makedirs(paths_segmentation, exist_ok=True)
                if os.listdir(paths_segmentation) == []:
                    shutil.move(dataset_train, os.path.join(paths_segmentation, "YOLODataset_train"))
                    shutil.move(dataset_val, os.path.join(paths_segmentation, "YOLODataset_val"))
                    print(f'Carpetas movidas exitosamente a {paths_segmentation} ')
                else:
                    print(f'Las carpetas ya se encuentran en {paths_segmentation} ')
            except OSError as e:
                print(f'Error al intentar crear la carpeta en {paths_segmentation}: {e}')
        return paths_segmentation  
    except TypeError as e:
        print(f'Tipo de dato incorrecto para el argumento: {e}')

detect_routes= acomodar_rutas_dataset(main_staticos, yolo_train_data, yolo_val_data)

In [ ]:
# Set de funciones para ordenar la estructura de los dataset individuales.

def arrange_dataset_paths(train, val):
    destination = os.path.abspath(os.path.join(train, "../../"))
    shutil.move(train, destination)
    if "dataset.yaml" in os.listdir(os.path.join(val, "../")):
        os.remove(val)

# Función para reescribir el dataset
def rewrite_dataset(dir_train_converted, combined_list):
    with open(dir_train_converted, 'w') as new_dataset:
        new_dataset.write(
            f'train: {os.path.abspath(os.path.join(dir_train_converted, "../"))}\n'
            f'val: {os.path.abspath(os.path.join(dir_train_converted, "../../"))}\\YOLODataset_val\n'
            'test: \n'
            f'nc: {len(combined_list)}\n'
            f'names: {combined_list}\n'
        )
        
# Función para combinar los dos dataset generados
def total_labels(list1, list2, update_dataset_train):
    set1=set(list1)
    set2=set(list2)
    union_without_repeating = set1.union(set2)
    combined_list = list(union_without_repeating)
    rewrite_dataset(update_dataset_train, combined_list)

# Función para extraer los nombres de las etiquetas de los dos dataset separados
def extract_labels(main_route):
    try:
        route_dataset_train= os.path.abspath(main_route) + "\\YOLODataset_train\\dataset.yaml"
        route_dataset_val= os.path.abspath(main_route) + "\\YOLODataset_val\\dataset.yaml"
        
        with open(route_dataset_train, 'r') as file_train:
            data_train = yaml.safe_load(file_train)
            labels_train = data_train['names']
        with open(route_dataset_val,'r') as file_val:
            data_val= yaml.safe_load(file_val)
            labels_val= data_val['names']
        total_labels(labels_train, labels_val, route_dataset_train)
        
        return route_dataset_train, route_dataset_val
    except (TypeError, FileNotFoundError) as e:
        print(f'Error: {e}')

try:
    train, val= extract_labels(detect_routes)
    arrange_dataset_paths(train, val)
except TypeError as e:
    print(f'Error: {e}')

In [4]:
# Buscar la carpeta que contiene la configuración completa para el entrenamiento
data_train= os.path.abspath(main_staticos) + "\\data_afectacion\\dataset.yaml"

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Selección versión de modelo YOLOv8</span>

In [4]:
# Modificación de las rutas de las carpetas predeterminadas de ultralytics YOLO

def set_config_ultralytics(main):
    settings.update({'datasets_dir': f'{main}\\dist', 'weights_dir': f'{main}\\src', 'runs_dir': f'{main}\\src\\runs'})
set_config_ultralytics(main_folder)

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Modelo original</span>

### Tabla de variantes en la Segmentación de instancias con YOLOv8

| Modelo      | Nombres de archivo                                      | Identificador |
|-------------|-----------------------------------------------------|------------|
| YOLOv8-seg  | yolov8n-seg.pt yolov8s-seg.pt yolov8m-seg.pt yolov8l-seg.pt yolov8x-seg.pt | n, s, m, l, x |


In [ ]:
package_installation('gdown') # Librería que le permitira descargar las versiones de YOLOv8

In [3]:
def get_model_version():
    available_versions= ['n', 's', 'm', 'l', 'x'] # yolov8n.pt , yolov8s.pt , yolov8m.pt , yolov8l.pt , yolov8x.pt
    control= True
    while control:
        version_selection= input("Ingrese la versión del modelo que desea utilizar, entre las disponibles están: n, s, m, l, x. Advertencia: Si no desea instalar ninguna ingrese la palabra 'salir' ").lower() 
        if version_selection == "salir":
            id_version=None
            control=False
        elif version_selection in available_versions:
            id_version= f'yolov8{version_selection}.pt' 
        else:
            print('Version de modelo inexistente, ingrese nuevamente! ')
        return id_version
     
def model_installation(url, version_name):
    try:
        path_save = os.path.abspath(main_folder) + "\\dist\\model_version\\" + version_name
        if os.path.exists(path_save):
            print(f'Se está utilizando la versión "{version_name}" del modelo. ')
        else:
            url_installation = f'{url}/{version_name}'
            gdown.download(url_installation, path_save, quiet=False)
            print('Versión instalada correctamente!')
    except (NameError, UnboundLocalError) as e:
        print(f'Error en la instalación de la versión del modelo: {e}')
    return path_save

model_url= 'https://github.com/ultralytics/assets/releases/download/v0.0.0' 
model_version= get_model_version()

if model_version is not None:
    model_orig= model_installation(model_url, model_version) 
else:
    print('Ha salido con exito! ')

Se está utilizando la versión "yolov8x.pt" del modelo. 


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Función selección modelos pre-entrenados</span>

In [4]:
# Función para obtener el último entrenamiento

def get_last_training(dir_src, det_type):
    try:
        yolo_model_folders= os.listdir(dir_src)
        for folder_name in yolo_model_folders:
            if folder_name == det_type:
                base_path = os.path.abspath(os.path.join(dir_src, folder_name))
                files= os.listdir(base_path)
                last_experiment = sorted(files)[-1]
                model= os.path.join(base_path, last_experiment, "weights", "best.pt")
                print(f'Los pesos del experimento que se van a utilizar son {model}')
                return model
        else:
            print(f'No existe ningún experimento de entrenamiento en la carpeta {dir_src}')
            return None
    except FileNotFoundError as e:
        print(f'Error: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Modelo entrenado</span>

In [5]:
# Llamado para obtener el último entrenamiento del modelo deteccion

version_afectation= "srcv5" # Nombre de la carpeta donde se encuentra el modelo pre-entrenado a usar
detection_type_afectation= "deteccion" # Establece el modo de detecciones de Yolo (deteccion o segmentación)
dir_model_version_afectation=os.path.abspath(os.path.join(main_production, version_afectation)+ "\\modelo_deteccion")
model_detection= get_last_training(dir_model_version_afectation, detection_type_afectation)

Error: [WinError 3] El sistema no puede encontrar la ruta especificada: 'c:\\Users\\matrix\\pruebayolo\\proyecto_yolo\\src\\srcv5\\modelo_deteccion'


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Instalación de recursos</span>

In [19]:
package_installation('ultralytics') # Instalación de la biblioteca Ultralytics
#ultralytics.checks()

El paquete 'ultralytics' ya está instalado.


In [5]:
#Función para determinar si tiene GPU Cuda para instalación de PYTorch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'CUDA: {torch.cuda.get_device_name(0)}')
else:
    try:
        command = 'wmic path win32_videocontroller get caption'
        device = subprocess.check_output(command, shell=True, universal_newlines=True)
        print(f'Tarjeta Grafica: {device.strip()}.\n \nAdvertencia: Debe utilizar la CPU para instalación de PyTorch! ')
    except subprocess.CalledProcessError as e:
        print(f'Error al ejecutar el comando: {e}')

Tarjeta Grafica: Caption                  

AMD Radeon(TM) Graphics.
 
Advertencia: Debe utilizar la CPU para instalación de PyTorch! 


<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Dirección para instalar mediante el comando la versión de PyTorch en función de los requerimientos computacionales: </span>[Versión PyTorch](https://pytorch.org/get-started/locally/) 

In [ ]:
command_pytorch = 'pip3 install torch torchvision torchaudio' # Comando de instalación según la version personalizada a sus requerimientos computacionales
substrings = command_pytorch.split(" ")
try:
    subprocess.check_call(substrings)
except subprocess.CalledProcessError as e:
    print(f'Error al instalar la version de PyTorch: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">ENTRENAMIENTO</span>

In [5]:
# --Inicio Menú
while True:
    try:
        model_type = input('Qué tipo de modelo desea utilizar, indique "original" o "entrenado": ').lower()
        if model_type == "original":
            model_detection= YOLO(model_orig)
            break  # Sale del bucle si la entrada es válida
        elif model_type == "entrenado":
            model_detection= YOLO(model_orig) 
            break  # Sale del bucle si la entrada es válida
        else:
            print('Caracter no valido! Intente nuevamente.')
    except Exception as e:
        print(f'Error: {e}')
# --Finalización Menú

PROJECT='deteccion' # Le permite asignar un nombre al directorio de inicio que contendrá los experimentos de detección de objetos, y debe estar entre comillas; se recomienda no utilizar espacios en el nombre.
NAME='prueba_entrenamiento' # El nombre del experimento entrenamiento para deteccion de objetos debe ir entre comillas. Evite el uso de espacios al nombrar las carpetas; en su lugar, utilice algún formato de nombres como camelCase, snake_case o PascalCase.
TASK='detect' # Define la tarea principal que desea realizar con el modelo YOLO v8; para este caso, la deteccion de objetos implica identificar la ubicación y la clase de objetos en una imagen o un flujo de video.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada. Puede especificarlo como un número entero, como imgsz a 640 para obtener un cuadrado perfecto, o como una tupla, como imgsz=(640,480) para establecer dimensiones específicas de ancho y alto. Se recomienda ajustar este valor según el tamaño del objeto que desea detectar. Para la detencción de objetos pequeños, se recomienda aumentar el valor a más de 640 píxeles para obtener una resolución más alta.
DATA=data_train  #  Le permite indicar la ruta al archivo que contiene los metadatos que utilizara el modelo detección de objetos y su configuración en formato YAML. Si especifica el valor data=None, el conjunto de datos coco128-seg.yaml se utiliza de forma predeterminada; de lo contrario, escriba la ruta al archivo YAML entre comillas utilizando barras diagonales (/) en lugar de barras invertidas (\).
EPOCHS=30 # Establezca el número de épocas del modelo YOLO v8 en la tarea de deteccion. Este valor representa el número total de iteraciones en todo el conjunto de datos de entrenamiento. Se recomienda experimentar con este parámetro dependiendo de la cantidad de imágenes disponibles. Si tiene un conjunto de datos grande, considere aumentar este valor por encima de 30 para obtener mejores resultados. Por otro lado, establecer epochs=None hará que el modelo continúe entrenándose hasta que la pérdida de validación deje de mejorar.
BATCH=-1 # Define la cantidad de imágenes procesadas simultáneamente en una iteración del modelo YOLO v8 en la deteccion. El valor predeterminado es 16; se recomienda establecerlo en -1 para aprovechar AutoBatch, que ajusta automáticamente el tamaño del lote para optimizar el rendimiento, evitar problemas de memoria y maximizar la eficiencia del entrenamiento. Si desea personalizarlo, exprese el valor del parámetro como un número entero.
OPTIMIZER='auto' # Define el algoritmo de optimización para el modelo de deteccion YOLO. Su elección ajusta los pesos del modelo durante el entrenamiento y es crucial para la velocidad y rendimiento. Puede tomar valores como 'SGD', 'Adam', 'Adamax', 'AdamW', 'NAdam', 'RAdam', 'RMSProp' y 'auto', este último selecciona automáticamente el optimizador más adecuado a la tarea segmentación de objetos.
WORKERS=1 # Especifica el número de hilos de trabajo para la carga de datos en el modelo deteccion YOLO. Es recomendable utilizar un número de subprocesos que se ajusten al número de núcleos del CPU disponibles en el sistema.
DEVICE= 'cpu' # Especifica el dispositivo de ejecución para la versión del modelo yolov8 en la operación de deteccion. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
PLOTS=True # Utilice valores booleanos (Verdadero o Falso) para controlar la generación de gráficos que permite visualizar y monitorear la pérdida y la precisión durante el entrenamiento de deteccion de objetos. Establecer plots=True activara la función; si desea desactivarla, establezca el valor del hiperparámetro en False.
SAVE=True # Cuando se establece en True, el modelo guarda puntos de control periódicamente durante el entrenamiento de deteccion. Se recomienda tener cuidado al establecer este valor en Verdadero, ya que está relacionado con el hiperparámetro SAVE_PERIOD; si establece el valor del hiperparametro a False, la función Save_Period se desactivará.
SAVE_PERIOD=-1 # Se utiliza para especificar con qué frecuencia se guardan los puntos de control durante el entrenamiento de deteccion. Si se establece en un valor mayor que 0, el modelo guardará puntos de control cada número especificado de épocas. Sin embargo, si save_period se establece en -1, significa que la función esta deshabilitada.
PATIENCE=30 # Representa el número esperado de épocas durante el entrenamiento de deteccion. Si no se observa mejora en el conjunto de validación dentro de un período específico, se detiene el proceso. Esta técnica de parada temprana se utiliza para evitar el sobreajuste del modelo. Se recomienda ajustar este hiperparámetro en función de la duración esperada del entrenamiento.
VERBOSE=True # Se utiliza para controlar el número de impresiones durante la ejecución del entrenamiento de deteccion. Para suprimir la salida de información básica únicamente, debe establecer el valor del hiperparámetro en False, pero si desea una salida de progreso más detallada, establezca el valor en True.
RECT= False # Habilita la formación rectangular en cada lote, redimensionando las imágenes para que todas tengan la misma forma rectangular. Puedes establecerlo en True si tu conjunto de datos es extenso y deseas acelerar el tiempo de entrenamiento en la deteccion de objetos. De lo contrario, si se establece en False el modelo se entrena en el orden normal procesando todos los datos de un lote antes de pasar al siguiente lote.
COS_LR=False # Reemplaza el decaimiento escalonado predeterminado de YOLOv8, que reduce la tasa de aprendizaje en ciertas épocas, con el decaimiento escalonado cos_lr. Este ajusta la tasa de aprendizaje según las épocas restantes y la tasa de aprendizaje inicial, proporcionando una disminución más suave. Establezca este hiperparámetro en True para una reducción gradual de la tasa de aprendizaje, de lo contrario establezca en False.
FRACTION= 1.0 # Controla la fracción del conjunto de datos que se utilizara para el entrenamiento de deteccion. Debes establecer este parámetro entre 0.0 y 1.0. Por defecto, cuando es 1.0, se emplea el 100% de las imágenes disponibles en el conjunto de datos.
EXIST_OK=False # Controla la sobreescritura de un experimento de deteccion existente. Cuando se establece en False, el sistema no sobrescribirá, en su lugar devolverá una ruta incrementada. Esto es útil para prevenir la sobreescritura accidental de experimentos anteriores. Para activar la función, asigna el valor True.
try:
    model_detection.train(project=PROJECT,name=NAME, task=TASK, data=DATA, imgsz=IMGSZ, epochs=EPOCHS, batch=BATCH, workers=WORKERS, device=DEVICE, plots=PLOTS, verbose=VERBOSE, rect=RECT, cos_lr=COS_LR, optimizer=OPTIMIZER, fraction=FRACTION, patience=PATIENCE, exist_ok=EXIST_OK)
except Exception as e:
    print(f'Error: {e}')

New https://pypi.org/project/ultralytics/8.1.17 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=c:\Users\matrix\pruebayolo\proyecto_yolo\dist\model_version\yolov8n.pt, data=c:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_afectacion\dataset.yaml, epochs=30, patience=30, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=1, project=deteccion, name=prueba_entrenamiento, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize

train: Scanning C:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_afectacion\YOLODataset_train\labels.cache... 199 images, 0 backgrounds, 0 corrupt: 100%|██████████| 199/199 [00:00<?, ?it/s]
val: Scanning C:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_afectacion\YOLODataset_val\labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]


Plotting labels to deteccion\prueba_entrenamiento\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to deteccion\prueba_entrenamiento
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30         0G      2.077       3.57      1.761        100        640: 100%|██████████| 13/13 [01:26<00:00,  6.67s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.97s/it]

                   all         49        235     0.0048      0.399     0.0596     0.0184


...

      30/30         0G       1.56      1.796      1.338         35        640: 100%|██████████| 13/13 [01:14<00:00,  5.74s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.22s/it]

                   all         49        235      0.256      0.274      0.182     0.0723



30 epochs completed in 0.720 hours.
Optimizer stripped from deteccion\prueba_entrenamiento\weights\last.pt, 6.2MB
Optimizer stripped from deteccion\prueba_entrenamiento\weights\best.pt, 6.2MB

Validating deteccion\prueba_entrenamiento\weights\best.pt...
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.62s/it]


                   all         49        235      0.305      0.288      0.187     0.0793
            abolladura         49         46      0.276       0.37      0.217      0.092
                 rayon         49        189      0.334      0.206      0.158     0.0666
Speed: 2.7ms preprocess, 83.4ms inference, 0.0ms loss, 7.8ms postprocess per image
Results saved to deteccion\prueba_entrenamiento


<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">Configuración para la exportación</span>


A continuación se muestra una tabla de referencia para exportar un modelo YOLOv8 entrenado en la tarea deteccion de objetos. Tenga en cuenta que configurar el parámetro de `format` es fundamental para el proceso de exportación. Antes de continuar, asegúrese de verificar y ajustar estos valores a sus requisitos específicos: 
| Formatos                                                             | Asignación | Extensión                     | Hyperparámetros                                            | Descripción                                        |
|--------------------------------------------------------------------|-------------------|---------------------------|-----------------------------------------------------|----------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                    | -                 | `yolov8n.pt`              | -                                                   | Modelo en formato PyTorch                           |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)            | "torchscript"     | `yolov8n.torchscript`     | `imgsz`, `optimize`                                 | Simplifica la implementación de modelos PyTorch en entornos de producción y aplicaciones eficientes, mejorando la portabilidad y el rendimiento al permitir ejecutar una representación intermedia en entornos sin Python.                       |
| [ONNX](https://onnx.ai/)                                           | "onnx"            | `yolov8n.onnx`            | `imgsz`, `half`, `dynamic`, `simplify`, `opset`     | Desarrollado para promover la interoperabilidad, la optimización del hardware y la colaboración entre comunidades, al tiempo que responde a la necesidad de portabilidad de los modelos entre distintos marcos y herramientas de aprendizaje automático.                              |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)             | "openvino"        | `yolov8n_openvino_model/` | `imgsz`, `half`, `int8`                             | Elaborado para promover la interoperabilidad, la optimización del hardware y el despliegue eficiente de modelos a través de diferentes marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware Intel.                     |
| [TensorRT](https://developer.nvidia.com/tensorrt)                  | "engine"          | `yolov8n.engine`          | `imgsz`, `half`, `dynamic`, `simplify`, `workspace` | Permite promover la interoperabilidad, la optimización del hardware y la implantación eficiente de modelos en distintos marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware de NVIDIA.                     |
| [CoreML](https://github.com/apple/coremltools)                     | "coreml"          | `yolov8n.mlpackage`       | `imgsz`, `half`, `int8`, `nms`                      | Posibilita promover la interoperabilidad, la optimización del hardware y el despliegue eficiente de modelos a través de diferentes marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware de Apple.                            |
| [TF SavedModel](https://www.tensorflow.org/guide/saved_model)      | "saved_model"     | `yolov8n_saved_model/`    | `imgsz`, `keras`, `int8`                            | Empleado para guardar, compartir y desplegar modelos entrenados con TensorFlow. Versátil y facilita el despliegue en diversas plataformas como servidores, dispositivos móviles, embebidos y navegadores.                     |
| [TF Lite](https://www.tensorflow.org/lite)                         | "tflite"          | `yolov8n.tflite`          | `imgsz`, `half`, `int8`                             | Diseñado para el aprendizaje automático en dispositivos, TF Lite aborda restricciones clave como latencia, privacidad, conectividad, tamaño y consumo de energía. Es esencial para desplegar modelos en dispositivos móviles e integrados, ofreciendo una solución ligera y eficiente.                           |
| [TF Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | "edgetpu"         | `yolov8n_edgetpu.tflite`  | `imgsz`                                             | Utilizado para desplegar modelos de aprendizaje automático en el Edge TPU de TensorFlow. El Edge TPU es un pequeño ASIC (Circuito Integrado Específico de Aplicación) diseñado por Google para ofrecer inferencias de aprendizaje automático de alto rendimiento en dispositivos de bajo consumo.                       |
| [TF.js](https://www.tensorflow.org/js)                             | "tfjs"            | `yolov8n_web_model/`      | `imgsz`                                             | Facilita el despliegue de modelos de aprendizaje automático en navegadores web y Node.js, destacando la portabilidad y la facilidad de uso.                    |
| [PaddlePaddle](https://github.com/PaddlePaddle)                    | "paddle"          | `yolov8n_paddle_model/`   | `imgsz`                                             | Utilizado para desplegar modelos en PaddlePaddle, una plataforma de aprendizaje profundo de código abierto, paralela y distribuida que tiene su origen en la práctica industrial.                      |
| [ncnn](https://github.com/Tencent/ncnn)                            | "ncnn"            | `yolov8n_ncnn_model/`     | `imgsz`, `half`                                     | Formato optimizado para plataformas móviles, ofreciendo alto rendimiento. Puede incluir una estructura de archivo de modelo con información sobre capas, blobs de entrada y salida, y otros parámetros.

<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">EXPORTACIÓN</span>

In [ ]:
selected_model_det=YOLO(model_detection)

FORMAT='onnx' # Seleccione el formato de exportación del modelo deteccion, empleando la tabla previamente proporcionada; en la columna "Asignación" se proporcionan las opciones para ajustar este valor.
INT8=True # Establezca este parámetro en True al utilizar la CPU y en False en caso contrario. La cuantificación a INT8 mejora la eficiencia del modelo deteccion en cuanto a memoria y velocidad de inferencia, especialmente en hardware que admite esta precisión.
HALF=False # Configúrelo en True cuando use la GPU; en caso contrario, False. La cuantificación a FP16 mejora la eficiencia de la memoria del modelo deteccion y la velocidad de inferencia, especialmente en hardware que admite precisión de punto flotante de 16 bits.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada para la exportación del modelo de deteccion. Puede especificarlo como un número entero, por ejemplo, 640 para un cuadrado perfecto, o como una tupla, por ejemplo, (640, 480) para dimensiones específicas de ancho y alto. Las imágenes que ingreses al modelo después de la exportación deben tener las mismas dimensiones específicas que has configurado para adaptarse a los requisitos del escenario de despliegue.
OPTIMIZE=False # Controla la optimización en modelos de detección a TorchScript para su implementación móvil. Es importante destacar que esta función puede resultar en un aumento significativo en el tamaño del modelo exportado, lo cual puede no ser ideal para aplicaciones móviles. Se configura con True para activar y False para desactivar.
DYNAMIC=False # Controla la habilitación de ejes dinámicos en modelos de deteccion, lo cual es particularmente útil para gestionar tamaños de lote variables. Esta característica funciona bien en escenarios donde el tamaño del lote puede cambiar durante la inferencia, como aplicaciones en tiempo real o de transmisión por secuencias. Los valores aceptados son Verdadero para habilitar la función y Falso para deshabilitarla.
SIMPLIFY=False # En la exportación de modelos de deteccion a ONNX|TensorRT, este hiperparámetro personaliza la complejidad del modelo controlando la optimización, eliminando capas redundantes y reduciendo la precisión de los parámetros. Se activa con True y se desactiva con False.
OPSET=False # Especifica la versión del conjunto de operadores en ONNX al exportar el modelo deteccion desde marcos como PyTorch o TensorFlow. Si se deja en "None", ONNX utilizará automáticamente la versión más reciente disponible; para una versión específica, asigne el número entre comillas, por ejemplo, "11".
WORKSPACE=4 # En la exportación de modelos de deteccion a TensorRT, establece el tamaño del espacio de trabajo en GB asignado para optimizar y preparar el modelo de red neuronal. Este espacio se utiliza durante la construcción del motor para lograr una ejecución eficiente en hardware GPU mediante la biblioteca TensorRT.
NMS=False # En la exportación de modelos de deteccion a CoreML, controla la inclusión de la Supresión No Máxima (NMS) en el modelo para eliminar cuadros delimitadores redundantes en la segmentación de instancias y mejorar la precisión de las predicciones. Establecer 'NMS' en 'False' ignora NMS en los modelos CoreML exportados.  Este ajuste, configurable durante la exportación del modelo YOLO, lo que permite a los usuarios optimizar la implementación del modelo en una variedad de plataformas y dispositivos.
KERAS= False # En la exportación de modelos de deteccion a TF SavedModel y TF Lite, permite optimizar el despliegue en diversas plataformas y dispositivos. Incluye también el formato del archivo, el dispositivo de ejecución y la posibilidad de manejar múltiples etiquetas por caja. Establezca el valor del hiperparámetro en True si está familiarizado con Keras; de lo contrario, en False para excluir su uso en la exportación.

selected_model_det.export(format=FORMAT, imgsz=640, dynamic=False, simplify=False, opset=False)


<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">VALIDACIÓN</span>

In [16]:
NAME='Prueba_validación_det' # El nombre del experimento de validación para deteccion de objetos deben ir entre comillas. Evite el uso de espacios al nombrar las carpetas; en su lugar, utilice algún formato de nombres como camelCase, snake_case o PascalCase.
DATA= data_train #Permite indicar la ruta al archivo que contiene los metadatos necesarios para el proceso de validación, la ruta debe ser proporcionada entre comillas.
IMGSZ=640 # Establece las dimensiones en píxeles de la imagen de entrada para la validación del modelo de deteccion. Puede ser un número entero, como 640 para un cuadrado perfecto, o una tupla, como (640, 480), para dimensiones específicas de ancho y alto. Se recomienda usar el mismo valor que utilizo durante el entrenamiento del modelo.
BATCH=16 # Define la cantidad de imágenes procesadas simultáneamente en una iteración para la validación en detecciones de objetos. El valor predeterminado es 16; se recomienda establecerlo en -1 para utilizar AutoBatch, que ajusta automáticamente el tamaño del lote para optimizar el rendimiento y la eficiencia del entrenamiento, evitando problemas de memoria. Si desea personalizarlo, establezca el valor como un número entero.
SAVE_HYBRID=True # Activa la función con True para guardar una versión híbrida de la etiqueta, incluyendo la original y predicciones adicionales. Útil para el análisis detallado del rendimiento del modelo deteccion durante la validación; establezca en False para mostrar solo las predicciones.
CONF=0.5 # Establece el umbral de confianza para la validación de clases en la tarea de deteccion. Se recomienda un valor entre 0.5 y 0.10. Un umbral más alto mejora la precisión pero reduce la frecuencia de predicciones, mientras que un umbral más bajo aumenta la frecuencia pero disminuye la precisión. 
MAX_DET=10 # Toma como valor solo números enteros. Índica el límite de la cantidad máxima de objetos que el modelo intentara detectar en una imagen. Se recomienda establecer un valor alto para evitar perder detecciones relevantes.
DEVICE='CPU' # Especifica el dispositivo de ejecución para la prueba de validación en la operación de deteccion. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
PLOTS=True # Utilice valores booleanos (Verdadero o Falso) para controlar la generación de gráficos que permite visualizar y monitorear la pérdida y la precisión durante la validación en la deteccion de objetos. Establecer plots=True activara la función; si desea desactivarla, establezca el valor del hiperparámetro en False.
RECT=False # Habilita la formación rectangular en cada lote, redimensionando las imágenes para que todas tengan la misma forma rectangular. Puedes establecerlo en True si tu conjunto de datos es extenso y deseas acelerar el tiempo de validación en la deteccion de objetos. De lo contrario, si se establece en False el modelo se entrena en el orden normal procesando todos los datos de un lote antes de pasar al siguiente.
IOU=0.6 # El umbral predeterminado para la supresión no máxima (NMS) en la validación YOLO es 0,6. Este umbral de IoU (intersección sobre unión) es fundamental para NMS porque determina el grado mínimo de superposición requerido para que dos cuadros delimitadores se consideren el mismo objeto. Un umbral de IoU más bajo hace que NMS sea más conservador, mientras que un umbral de IoU más alto permite que un NMS más relajado evite eliminar los verdaderos positivos.

# --Inicio Menú
while True:
    try:
        model_type = input('Qué tipo de modelo desea utilizar, indique "original" o "entrenado": ').lower()
        if model_type == "original":
            selected_model_det=YOLO(model_orig)
            break  # Sale del bucle si la entrada es válida
        elif model_type == "entrenado":
            selected_model_det=YOLO(model_detection)
            break  # Sale del bucle si la entrada es válida
        else:
            print('Caracter no valido! Intente nuevamente.')
    except Exception as e:
        print(f'Error: {e}')
# --Finalización Menú

try:
    selected_model_det.val(name=NAME, data=DATA, imgsz=IMGSZ, batch=BATCH, save_hybrid=SAVE_HYBRID, conf=CONF, max_det=MAX_DET, device=DEVICE, plots=PLOTS, rect=RECT, iou=IOU, save_json=True)
except Exception as e:
    print(f'Error: {e}')

Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_afectacion\YOLODataset_val\labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:06<00:00,  1.73s/it]


                   all         49        235          1      0.865       0.93       0.93
            abolladura         49        189          1       0.73      0.865      0.865
                 rayon         49         46          1          1      0.995      0.995
Speed: 3.2ms preprocess, 114.6ms inference, 0.0ms loss, 0.6ms postprocess per image
Saving c:\Users\matrix\pruebayolo\proyecto_yolo\src\runs\detect\Prueba_validación_det\predictions.json...
Results saved to c:\Users\matrix\pruebayolo\proyecto_yolo\src\runs\detect\Prueba_validación_det


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Configuración de fuentes</span>

Para utilizar múltiples fuentes de datos al realizar predicciones con el modelo, se requiere que se ajuste el parámetro'source' a sus necesidades, tal como se indica en la siguiente tabla:
| Fuentes          | Asignación                             | Tipo             | Notas                                                           |
| --------------- | ------------------------------------ | ----------------- | --------------------------------------------------------------- |
| `image`           | 'image.jpg'                          | str or Path       | Archivo que contiene una única imagen.                                              |
| `URL`             | 'https://ultralytics.com/images/bus.jpg' | str               | Dirección que especifica la ubicación de una imagen en la web.                                                 |
| `screenshot`      | 'screen'                             | str               | El sistema captura la imagen actualmente visible en la pantalla y la utiliza como entrada para el modelo.                                           |
| `PIL`             | Image.open('im.jpg')                 | PIL.Image         | Utilizado para cargar imágenes en formato HWC (altura, ancho, canales) con canales RGB (rojo, verde y azul) mediante la biblioteca Python Imaging Library (PIL).                                   |
| `OpenCV`          | cv2.imread('im.jpg')                 | np.ndarray        | Permite la lectura de una imagen desde un archivo en formato HWC con canales BGR (azul, verde, rojo) utilizando la biblioteca OpenCV, almacenando la imagen como un array de NumPy.                    |
| `numpy`           | np.zeros((640,1280,3))               | np.ndarray        | Genera un array de ceros con las dimensiones especificadas para un formato HWC con canales BGR, utilizando la biblioteca NumPy.                    |
| `torch`           | torch.zeros(16,3,320,640)            | torch.Tensor      | Crea un tensor de ceros con las dimensiones especificadas para un formato HWC con canales RGB, empleando el framework PyTorch.               |
| `CSV`             | 'sources.csv'                        | str or Path       | Archivo de texto que almacena las rutas a las imágenes que se procesarán.   |
| `video`          | 'video.mp4'                          | str or Path       | Proporciona acceso a un archivo de video único.                       |
| `directory`      | 'path/'                              | str or Path       | Directorio que contiene múltiples archivos de imagen.               |
| `glob`           | 'path/*.jpg'                         | str               | Permite acceder a varias imágenes en un directorio usando expresiones de coincidencia de patrones. |
| `YouTube`        | 'https://youtu.be/LNwODJXcvt4'       | str               | Facilita el acceso a videos desde la plataforma YouTube.                                         |
| `stream`         | 'rtsp://example.com/media.mp4'      | str               | Permite la conexión a flujos de video o audio en tiempo real mediante protocolos como RTSP, RTMP, TCP o IP, ya sea a través de internet o una red local. |
| `multi-stream`   | 'list.streams'                       | str or Path       | Se utiliza para transmitir varios flujos de medios simultáneamente, permitiendo el procesamiento y análisis paralelo de múltiples flujos de medios. |

Formatos para las imágenes: 

| Image Suffixes | Reference                           |
| --------------- | ----------------------------------- |
| .bmp            | [Microsoft BMP File Format](https://docs.fileformat.com/es/image/bmp/)           |
| .dng            | [Adobe DNG](https://docs.fileformat.com/es/image/dng/)                           |
| .jpeg           | [JPEG](https://docs.fileformat.com/es/image/jpeg/)                                |
| .jpg            | [JPEG](https://docs.fileformat.com/es/image/jpeg/)                                |
| .mpo            | [Multi Picture Object](https://docs.fileformat.com/es/image/mpo/)                |
| .png            | [Portable Network Graphics](https://docs.fileformat.com/es/image/png/)           |
| .tif            | [Tag Image File Format](https://docs.fileformat.com/es/image/tiff/)               |
| .tiff           | [Tag Image File Format](https://docs.fileformat.com/es/image/tiff/)               |
| .webp           | [WebP](https://docs.fileformat.com/es/image/webp/)                                |
| .pfm            | [Portable FloatMap](https://docs.fileformat.com/font/pfm/)                   |


Formatos para los videos: 

| Video Suffixes | Reference                           |
| -------------- | ----------------------------------- |
| .asf           | [Advanced Systems Format](https://docs.fileformat.com/es/video/asf/)             |
| .avi           | [Audio Video Interleave](https://docs.fileformat.com/es/video/avi/)              |
| .gif           | [Graphics Interchange Format]()          |
| .m4v           | [MPEG-4 Part 14](https://docs.fileformat.com/es/video/m4v/)                      |
| .mkv           | [Matroska](https://docs.fileformat.com/es/video/mkv/)                            |
| .mov           | [QuickTime File Format](https://docs.fileformat.com/es/video/mov/)               |
| .mp4           | [MPEG-4](https://docs.fileformat.com/es/video/mp4/)          |
| .mpeg          | [MPEG-1](https://docs.fileformat.com/es/video/mpeg/)                       |
| .mpg           | [MPEG-1](https://docs.fileformat.com/es/video/mpeg/)                       |
| .ts            | [MPEG Transport Stream](https://docs.fileformat.com/es/video/ts/)               |
| .wmv           | [Windows Media Video](https://docs.fileformat.com/es/video/wmv/)                 |
| .webm          | [WebM Project](https://docs.fileformat.com/es/video/webm/)      

In [20]:
def convert_path(path): 
    return os.path.normpath(path).replace(os.sep, '/')

convert_path(r'C:\Users\matrix\pruebayolo\proyecto_yolo\src\prueba')

'C:/Users/matrix/pruebayolo/proyecto_yolo/src/prueba'

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">PREDICCIÓN</span>

In [7]:
SOURCE= dir_prueba # Establezca el origen de datos que el modelo de detección utilizará para realizar predicciones. Configure el valor de este hiperparámetro según la tabla proporcionada anteriormente.
MAX_DET=3 # Toma como valor solo números enteros. Índica el límite de la cantidad máxima de objetos que el modelo intentara predecir en una imagen. Se recomienda establecer un valor alto para evitar perder detecciones relevantes.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada durante la predicción en tareas de detección. Puede ser un número entero, como 640 para un cuadrado perfecto, o una tupla, como (640, 480), para dimensiones específicas de ancho y alto. Se recomienda utilizar los mismos valores utilizados durante el entrenamiento del modelo para mantener la coherencia en la inferencia.
CONF=0.5 # Establece el umbral de confianza durante el proceso de predicción en la tarea de detección. Se recomienda establecer el valor hiperparámetro entre 0.5 y 0.10. Un umbral más alto mejora la precisión pero reduce la frecuencia de predicciones, mientras que un umbral más bajo aumenta la frecuencia pero disminuye la precisión en la inferencia.
LINE_WIDTH= None # Determina el grosor en píxeles de los cuadros delimitadores que rodean los objetos detectados por el modelo. Puede establecer el grosor de la línea como un número entero en el que, a mayor valor, la línea será más gruesa, también puede utilizar como valor None para que el grosor se ajuste de forma automatizada, proporcionando una línea proporcional al tamaño de la imagen.
VISUALIZE=False # Determina si las características del modelo de detección deben mostrarse durante la predicción. Establecer esto en True permite que las características se muestren como mapas intermedias, lo que hace que el modelo sea más fácil de entender. Si se establece en False, no se mostrarán las características del modelo. 
IOU=0.7 # El umbral predeterminado para la supresión no máxima (NMS) en la validación YOLO es 0,7. Este umbral de IoU (intersección sobre unión) es fundamental para NMS porque determina el grado mínimo de superposición requerido para que dos cuadros delimitadores se consideren la misma detección. Un umbral de IoU más bajo hace que NMS sea más conservador, mientras que un umbral de IoU más alto permite que un NMS más relajado evite eliminar los verdaderos positivos.
DEVICE='cpu' # Especifica el dispositivo de ejecución para la prueba de predicción en la operación de detección. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
VID_STRIDE=False # Controla la velocidad de los fotogramas durante el proceso de predicción en vídeos o secuencias de tiempo real. Al establecerlo en True el modelo se adapta a la velocidad de fotogramas especificada por la fuente de vídeo, procesando cada fotograma individualmente. Para desactivar esta función indique como valor False.
STREAM_BUFFER=False # Controla el almacenamiento en búfer de los fotogramas para la detección. Si es True, se almacenan todos los fotogramas para el procesamiento en tiempo real de vídeos o transmisiones en directo; si es False, devuelve el fotograma más reciente.
SAVE_FRAMES=False # Controla la captura y almacenamiento de los fotogramas predichos por el modelo de detección. Con True, se guardarán todos los fotogramas individuales predichos; con False, no se realizará el almacenamiento de los fotogramas.
AUGMENT=True # Aplica transformaciones a las imágenes de entrada, tales como giros, rotaciones, recortes y cambios de color, para diversificar los datos y mejorar la predicción en la detección. Establecer en True para activar la función, False para desactivar.
SAVE_CROP=False # Determina si se deben guardar imágenes recortadas con los resultados durante la predicción en la detección. Al establecerlo en "False", las imágenes recortadas no se guardarán, lo que reduce el tamaño del archivo. Con el valor "True", se guardarán las imágenes recortadas correspondientes a las áreas detectadas.
SHOW=False # Determina si se deben mostrar las imágenes o vídeos detectados durante la predicción. Al establecerlo en "True", permite la visualización de las predicciones en el mismo entorno, proporcionando una representación visual de los resultados. Si se establece en "False", las predicciones no se mostrarán. 
SAVE_TXT= False #
SAVE_CONF=True #
SAVE=True #

def thread_safe_predict(model, output_queue, id_class, conf_value, name_experiment, data, saved_verification, txt_verification, max_detection): #  Realiza una predicción segura utilizando un modelo YOLO.
    try:
        local_model = YOLO(model) # Inicializa el modelo YOLO.
        results = local_model.predict(source= data, save=saved_verification, classes=id_class, conf=conf_value, name=name_experiment, save_txt=txt_verification, max_det=max_detection, line_width=LINE_WIDTH, visualize=VISUALIZE, imgsz=IMGSZ, iou=IOU, device=DEVICE, vid_stride=VID_STRIDE, stream_buffer=STREAM_BUFFER, save_crop=SAVE_CROP, show=SHOW, save_frames=SAVE_FRAMES, save_conf=SAVE_CONF, retina_masks=True, agnostic_nms=True) # Se establecen los hiperpametros a utilizar en el proceso de predicción.
        output_queue.put((results)) # Pone los resultados en la cola.
        
    except Exception as e: # Atrapa cualquier mensaje de error. 
        print(f"Error in thread_safe_predict: {e}")

def delete_temp(temp_car): # Permite eliminar un directorio temporal.
    try: 
        shutil.rmtree(temp_car)  # Elimina un directorio temporal según la ruta especificada.
        print(f'Temporary directory {temp_car} successfully deleted!')
    except Exception as e: # Captura cualquier excepción e imprime un mensaje de error.
        print(f'Error deleting the directory: {e}')

output_queue = Queue()
try: 
    Thread(target=thread_safe_predict, args=(model_orig, output_queue, 2, 0.5, "predict_car", SOURCE, True, False, MAX_DET)).start() # Inicio de un nuevo hilo para la predicción del coche.
    results_car= output_queue.get() # Obtiene los resultados de la cola.
    dir_temp_car= f'{results_car[0].save_dir}' # Obtiene la ruta donde fueron guardados los resultados de la predicción vehículo.

    try:
        Thread(target=thread_safe_predict, args=(model_detection, output_queue, 0, 0.5, "predict_abolladura", dir_temp_car, SAVE, SAVE_TXT, 2)).start() # Inicia un nuevo hilo para la predicción de abolladura, la cual se basara en los resultados de vehiculo.
        results_abolladura = output_queue.get() 
        Thread(target=thread_safe_predict, args=(model_detection, output_queue, 1, 0.2, "predict_rayon", dir_temp_car, SAVE, SAVE_TXT, MAX_DET)).start() # Inicia un nuevo hilo para la predicción de rayón, la cual se basara en los resultados de vehiculo.
        results_rayon = output_queue.get() 
        delete_temp(dir_temp_car) # Elimina el directorio temporal para los resultados de los coches.
    except Exception as e:
        print(f"No vehicle images found: {e} ")
except Exception as e:
    print(f"Error in prediction by: {e} ") 
        


image 1/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\20170531_010133.jpg: 384x640 1 car, 1717.4ms
image 2/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\5-320w-320w.jpg: 480x640 1 car, 1654.0ms
image 3/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\Car-Dent.png: 224x640 1 car, 866.3ms
image 4/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\Charlotte-Toyota-service-1-1024x683.jpg: 448x640 1 car, 1477.7ms
image 5/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\GSDH75MG6FAM7FSG43RFQLN5R4.jpg: 448x640 1 car, 1632.6ms
image 6/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\OIP (1).jpg: 480x640 (no detections), 1786.0ms
image 7/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\OIP (2).jpeg: 640x640 (no detections), 2053.5ms
image 8/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\OIP (4).jpeg: 480x640 1 car, 1622.4ms
image 9/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\OIP(3).jpeg: 448x640 1 car, 1478.5ms


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Función de área para cajas detectadas</span>

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Dataframe con las áreas totales</span>

In [8]:
def calculate_area(mask):
    sum_area = 0
    for e in mask:
        area = torch.round(e[2] * e[3]).int()
        sum_area += area.sum().item()
    return sum_area
    
def search_area_by_instance(results, image_instance):
    for i in results:
        if os.path.basename(i.path) == image_instance:
            return calculate_area(i.boxes.xywh) if i.boxes is not None else 0
    return 0

def process_main_detection(dent, scratch, car_results):
    columnas = ['imagen','area abolladura', 'area rayon', 'area carro','afectacion']
    data= []
    
    for result in car_results:
        control_image= os.path.basename(result.path)

        dent_area  = search_area_by_instance(dent, control_image)
        scratch_area = search_area_by_instance(scratch, control_image)

        if dent_area > 0 or scratch_area > 0:
            car_area = search_area_by_instance(result, control_image)
            damage= process_secondary_detection(car_area, dent_area, scratch_area)
        else:
            car_area= 0
            damage= 0

        data.append([f'{os.path.basename(control_image)}', f'{dent_area}', f'{scratch_area}', f'{car_area}', f'{damage}px^2'])
        
    df=pd.DataFrame(data, columns=columnas) 
    return df  
    
def process_secondary_detection(car_area, dent_area, scratch_area):
    if dent_area > 0 and scratch_area > 0:
        main_area_sum = dent_area + scratch_area 
        total_affected_area = main_area_sum - car_area
    else:
        total_affected_area = abs(dent_area - car_area) if dent_area > 0 else abs(scratch_area - car_area) if scratch_area > 0 else 0 
    return total_affected_area

df_result = process_main_detection(results_abolladura, results_rayon, results_car)

styled_df = df_result.style.set_properties(**{'text-align': 'right'})
styled_df


,imagen,area abolladura,area rayon,area carro,afectacion
0,20170531_010133.jpg,0,52057,1230189,1178132px^2
1,5-320w-320w.jpg,25785,0,66753,40968px^2
2,Car-Dent.png,96449,0,150979,54530px^2
3,Charlotte-Toyota-service-1-1024x683.jpg,0,9851,597363,587512px^2
4,GSDH75MG6FAM7FSG43RFQLN5R4.jpg,0,408,642499,642091px^2
5,OIP (1).jpg,0,10388,0,10388px^2
6,OIP (2).jpeg,0,537,0,537px^2
7,OIP (4).jpeg,0,13612,762729,749117px^2
8,OIP(3).jpeg,0,9045,144130,135085px^2
9,R.jpeg,0,2859,0,2859px^2


# Dataframe con las áreas individuales

In [9]:
def calculate_area(segmented_masks): 
    sum_area = 0
    area_individual = 0
    if segmented_masks.numel() > 0:
        area_individual= []
        for e in segmented_masks:
            area = torch.round(e[2] * e[3]).int()
            area_individual.append(area.item())
            sum_area += area.sum().item()
    return area_individual, sum_area

def search_mask_by_image(segmentation_results, id_image):
    for attributes in segmentation_results:
        if os.path.basename(attributes.path) == id_image: # Confirmar que estamos buscando el resultado en la misma imagen por cada clase, con el atributo path obtenemos la direccion del archivo
            return calculate_area(attributes.boxes.xywh) if attributes.boxes is not None else ([], 0) # Envia las coordenadas para el calculo del area si la mascara no esta vacia. De lo contrario, retornara como valor una lista vacia referente a las areas halladas y un equivalente a cero por el area total de la imagen.
    return 0, 0


def process_segmentation_results(abolladura, rayon, car):
    columnas= ['imagen', 'ClaseMascara', 'area x mascara', 'afectacion x imagen']
    data= []

    for attribute in car: # Recorre los resultados de la segmentacion abolladura, una instancia obtendra el identificador de recorrido.
        img_directory = os.path.basename(attribute.path) # Se obtiene el atributo de directorio en imagen
            
        areas_abolladura, total_abolladura_area = search_mask_by_image(abolladura, img_directory)      
        areas_rayon, total_rayon_area = search_mask_by_image(rayon, img_directory)
            
        if total_abolladura_area is not None and total_abolladura_area > 0 or total_rayon_area is not None and total_rayon_area > 0:
            areas_car, total_car_area = search_mask_by_image(attribute, img_directory)
            afectacion_imagen= calculating_car_affectation(total_car_area, total_abolladura_area, total_rayon_area)
                
        else:
            areas_car= 0
            afectacion_imagen = 0
            
        data.append([img_directory, 'abolladura', areas_abolladura, afectacion_imagen])
        data.append([img_directory, 'rayon', areas_rayon, afectacion_imagen])
        data.append([img_directory, 'carro', areas_car, afectacion_imagen])
        
    orden_clases=['abolladura', 'rayon', 'carro']
    df= pd.DataFrame(data, columns=columnas)
    df['Clase'] = pd.Categorical(df['ClaseMascara'], categories=orden_clases, ordered=True)
    grouped_columns=df.groupby(['imagen', 'afectacion x imagen', 'Clase'], group_keys=True, as_index=True, observed=False)[['area x mascara']].apply(lambda x : x) 
    grouped_columns_expanded = grouped_columns.explode('area x mascara')
    return grouped_columns_expanded


def calculating_car_affectation(total_car_area, total_abolladura_area, total_rayon_area): 
    total_area_affected  = abs(total_car_area - total_abolladura_area) if total_abolladura_area is not None and total_abolladura_area > 0 else abs(total_car_area - total_rayon_area) if total_rayon_area is not None and total_rayon_area > 0 else 0 # verifica si existe un valor valido de abolladura para encontrar el valor de area afectada en el carro, de lo contrario verifica lo mismo para rayon, asigna como valor de afectacion 0 en caso de no ser valido.
    
    if total_abolladura_area is not None and total_rayon_area is not None and total_abolladura_area > 0 and total_rayon_area > 0:
        total_area_affected = total_abolladura_area + total_rayon_area - total_car_area # Si ambos tipos de afectacion existen en una imagen, se sumaran primero y restaran con el area del carro.
    return total_area_affected

dataframe = process_segmentation_results(results_abolladura, results_rayon, results_car)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
dataframe

area x mascara
imagen                                             afectacion x imagen Clase                       
20170531_010133.jpg                                 1178132            abolladura 0               0
                                                                       rayon      1           48466
                                                                                  1            2445
                                                                                  1            1146
                                                                       carro      2         1230189
5-320w-320w.jpg                                     40968              abolladura 3           25785
                                                                       rayon      4               0
                                                                       carro      5           66753
Car-Dent.png                                        54530              abolladura 6           96449
                                                                       rayon      7               0
                                                                       carro      8          150979
Charlotte-Toyota-service-1-1024x683.jpg             587512             abolladura 9               0
                                                                       rayon      10           2159
                                                                                  10           4552
                                                                                  10           3140
                                                                       carro      11         597363
GSDH75MG6FAM7FSG43RFQLN5R4.jpg                      642091             abolladura 12              0
                                                                       rayon      13            408
                                                                       carro      14         642499
OIP (1).jpg                                         10388              abolladura 15              0
                                                                       rayon      16           4406
                                                                                  16           4690
                                                                                  16           1292
                                                                       carro      17              0
OIP (2).jpeg                                        537                abolladura 18              0
                                                                       rayon      19            390
                                                                                  19            147
                                                                       carro      20              0
OIP (4).jpeg                                        749117             abolladura 21              0
                                                                       rayon      22           2247
                                                                                  22           8814
                                                                                  22           2551
                                                                       carro      23         762729
OIP(3).jpeg                                         135085             abolladura 24              0
                                                                       rayon      25           9045
                                                                       carro      26         144130
R.jpeg                                              2859               abolladura 27              0
                                                                       rayon      28           2859
                                                                       carro      29 

### Predicción | Visualización de clases en única imagen

In [6]:
SOURCE= dir_prueba # Establezca el origen de datos que el modelo de detección utilizará para realizar predicciones. Configure el valor de este hiperparámetro según la tabla proporcionada anteriormente.
MAX_DET=3 # Toma como valor solo números enteros. Índica el límite de la cantidad máxima de objetos que el modelo intentara predecir en una imagen. Se recomienda establecer un valor alto para evitar perder detecciones relevantes.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada durante la predicción en tareas de detección. Puede ser un número entero, como 640 para un cuadrado perfecto, o una tupla, como (640, 480), para dimensiones específicas de ancho y alto. Se recomienda utilizar los mismos valores utilizados durante el entrenamiento del modelo para mantener la coherencia en la inferencia.
CONF=0.5 # Establece el umbral de confianza durante el proceso de predicción en la tarea de detección. Se recomienda establecer el valor hiperparámetro entre 0.5 y 0.10. Un umbral más alto mejora la precisión pero reduce la frecuencia de predicciones, mientras que un umbral más bajo aumenta la frecuencia pero disminuye la precisión en la inferencia.
LINE_WIDTH= None # Determina el grosor en píxeles de los cuadros delimitadores que rodean los objetos detectados por el modelo. Puede establecer el grosor de la línea como un número entero en el que, a mayor valor, la línea será más gruesa, también puede utilizar como valor None para que el grosor se ajuste de forma automatizada, proporcionando una línea proporcional al tamaño de la imagen.
VISUALIZE=False # Determina si las características del modelo de detección deben mostrarse durante la predicción. Establecer esto en True permite que las características se muestren como mapas intermedias, lo que hace que el modelo sea más fácil de entender. Si se establece en False, no se mostrarán las características del modelo. 
IOU=0.7 # El umbral predeterminado para la supresión no máxima (NMS) en la validación YOLO es 0,7. Este umbral de IoU (intersección sobre unión) es fundamental para NMS porque determina el grado mínimo de superposición requerido para que dos cuadros delimitadores se consideren la misma detección. Un umbral de IoU más bajo hace que NMS sea más conservador, mientras que un umbral de IoU más alto permite que un NMS más relajado evite eliminar los verdaderos positivos.
DEVICE='cpu' # Especifica el dispositivo de ejecución para la prueba de predicción en la operación de detección. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
VID_STRIDE=False # Controla la velocidad de los fotogramas durante el proceso de predicción en vídeos o secuencias de tiempo real. Al establecerlo en True el modelo se adapta a la velocidad de fotogramas especificada por la fuente de vídeo, procesando cada fotograma individualmente. Para desactivar esta función indique como valor False.
STREAM_BUFFER=False # Controla el almacenamiento en búfer de los fotogramas para la detección. Si es True, se almacenan todos los fotogramas para el procesamiento en tiempo real de vídeos o transmisiones en directo; si es False, devuelve el fotograma más reciente.
SAVE_FRAMES=False # Controla la captura y almacenamiento de los fotogramas predichos por el modelo de detección. Con True, se guardarán todos los fotogramas individuales predichos; con False, no se realizará el almacenamiento de los fotogramas.
AUGMENT=True # Aplica transformaciones a las imágenes de entrada, tales como giros, rotaciones, recortes y cambios de color, para diversificar los datos y mejorar la predicción en la detección. Establecer en True para activar la función, False para desactivar.
SAVE_CROP=False # Determina si se deben guardar imágenes recortadas con los resultados durante la predicción en la detección. Al establecerlo en "False", las imágenes recortadas no se guardarán, lo que reduce el tamaño del archivo. Con el valor "True", se guardarán las imágenes recortadas correspondientes a las áreas detectadas.
SHOW=False # Determina si se deben mostrar las imágenes o vídeos detectados durante la predicción. Al establecerlo en "True", permite la visualización de las predicciones en el mismo entorno, proporcionando una representación visual de los resultados. Si se establece en "False", las predicciones no se mostrarán. 
SAVE_TXT= False #
SAVE_CONF=True #
SAVE=True #

def thread_safe_predict(model, output_queue, id_class, conf_value, name_experiment, data, saved_verification, txt_verification, max_detection): #  Realiza una predicción segura utilizando un modelo YOLO.
    try:
        local_model = YOLO(model) # Inicializa el modelo YOLO.
        results = local_model.predict(source= data, save=saved_verification, classes=id_class, conf=conf_value, name=name_experiment, save_txt=txt_verification, max_det=max_detection, line_width=LINE_WIDTH, visualize=VISUALIZE, imgsz=IMGSZ, iou=IOU, device=DEVICE, vid_stride=VID_STRIDE, stream_buffer=STREAM_BUFFER, save_crop=SAVE_CROP, show=SHOW, save_frames=SAVE_FRAMES, save_conf=SAVE_CONF, retina_masks=True, agnostic_nms=True) # Se establecen los hiperpametros a utilizar en el proceso de predicción.
        output_queue.put((results)) # Pone los resultados en la cola.
        
    except Exception as e: # Atrapa cualquier mensaje de error. 
        print(f"Error in thread_safe_predict: {e}")

def delete_temp(temp_car): # Permite eliminar un directorio temporal.
    try: 
        shutil.rmtree(temp_car)  # Elimina un directorio temporal según la ruta especificada.
        print(f'Temporary directory {temp_car} successfully deleted!')
    except Exception as e: # Captura cualquier excepción e imprime un mensaje de error.
        print(f'Error deleting the directory: {e}')

output_queue = Queue()
try: 
    Thread(target=thread_safe_predict, args=(model_orig, output_queue, 2, 0.5, "predict_car", SOURCE, True, False, MAX_DET)).start() # Inicio de un nuevo hilo para la predicción del coche.
    results_car= output_queue.get() # Obtiene los resultados de la cola.
    dir_temp_car= f'{results_car[0].save_dir}' # Obtiene la ruta donde fueron guardados los resultados de la predicción vehículo.

    try:
        Thread(target=thread_safe_predict, args=(model_detection, output_queue, [0, 1], 0.4, "predict_afectation_det_srcv5", dir_temp_car, SAVE, SAVE_TXT, MAX_DET)).start()  # Inicia un nuevo hilo para la predicción de afectación, la cual se basara en los resultados de vehiculo.
        results_afectation= output_queue.get() # Obtiene los resultados de la cola.
        delete_temp(dir_temp_car) # Eliminar el directorio temporal para los resultados de los coches.
    except Exception as e: # Captura de excepciones para la predicción de afectación.
        print(f"No vehicle images found: {e} ")
        
except Exception as e:
    print(f"Error in prediction by: {e} ") 
        


image 1/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\20170531_010133.jpg: 384x640 1 car, 1543.9ms
image 2/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\5-320w-320w.jpg: 480x640 1 car, 1192.5ms
image 3/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\Car-Dent.png: 224x640 1 car, 470.7ms
image 4/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\Charlotte-Toyota-service-1-1024x683.jpg: 448x640 1 car, 773.4ms
image 5/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\GSDH75MG6FAM7FSG43RFQLN5R4.jpg: 448x640 1 car, 715.8ms
image 6/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\OIP (1).jpg: 480x640 (no detections), 868.8ms
image 7/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\OIP (2).jpeg: 640x640 (no detections), 1338.3ms
image 8/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\OIP (4).jpeg: 480x640 1 car, 852.5ms
image 9/25 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_2\OIP(3).jpeg: 448x640 1 car, 887.9ms
image

In [41]:
def calculate_area(vertices):
    total_boxes_area = 0 # Inicializa una variable acumuladora para almacenar el área de cada caja delimitadora.
    for point in vertices: # Itera sobre cada conjunto de coordenadas en 'vertices' que tiene un formato: x [0](coordenada x), y[1](coordenada y), w[2](ancho), h[3](alto) de las cajas delimitadoras. 
        box_area = torch.round(point[2] * point[3]).int() # Calcula el área de la caja (ancho * alto) y lo redondea a un entero, obtiene como resultado un formato: tensor(1230189, dtype=torch.int32).
        total_boxes_area += box_area.sum().item() # Convierte el tensor 'box_area' a un valor entero y lo añade a la lista 'individual_boxes_area'.
    return total_boxes_area # Retorna la lista de áreas individuales de las cajas delimitadoras según la clase.

def search_for_mask(results, image_instance):
    for attribute in results: # Se itera sobre los resultados de la segmentación.
        if os.path.basename(attribute.path) == image_instance:
            # En los resultados de las máscaras cuando se manejan múltiples clases de objetos (hiperparámetro de predicción classes= []) en una imagen, es difícil referenciar las coordenadas que pertenecen a cada clase de objeto detectado, por lo tanto, es necesario separar dichos resultados de coordenadas. 
            if (attribute.boxes.cls != 2.).all():
                dent_coordinates = [] # Inicializa una lista temporal para las coordenadas de abolladura.
                scratch_coordinates = [] # Inicializa una lista temporal para las coordenadas de rayón.
                tensor = attribute.boxes.cls # Almacena los valores del tensor, que hacen referencia al cuadro delimitador.
                for position, value in enumerate(tensor): # Itera sobre los valores del tensor y las posiciones correspondientes.
                    if value == 0.: # Si el valor del tensor es 0, corresponde a una detección de abolladura.
                        dent_coordinates.append(attribute.boxes.xywh[position]) # Añade las coordenadas de la máscara a la lista de abolladuras.
                    elif value == 1.:  # Asegura que solo maneje dos clases de afectación.
                        scratch_coordinates.append(attribute.boxes.xywh[position]) # Añade las coordenadas de la máscara a la lista de rayon.
                dent_area, scratch_area = calculate_area(dent_coordinates), calculate_area(scratch_coordinates) # Calcula el área para cada clase de afectación.
                return dent_area, scratch_area 
            else: # Si los resultados que se estan recorriendo pertenecen a vehiculo, se retornara el area vehicular.
                return calculate_area(attribute.boxes.xywh) if attribute.boxes is not None else 0
    return 0 # Si no hay resultados para la imagen actual, retorna 0.

def process_main_detection(class_afectation, car_results): # Función main que controla todo el proceso.
    columnas = ['imagen','area abolladura', 'area rayon', 'area carro','afectacion'] # Se definen las columnas del DataFrame.
    data= [] # Se inicializa una lista vacía para almacenar los resultados de cada imagen.
    
    for result in car_results: # Se recorren principalmente los resultados de carro, ya que es el primer filtro por el que pasa el modelo de detección.
        img_directory= os.path.basename(result.path) # Extrae el nombre base de la imagen de control.

        dent_area, scratch_area = search_for_mask(class_afectation, img_directory) # Extrae las áreas de abolladuras y rayones en la imagen especificada.
        
        if dent_area > 0 or scratch_area > 0: # En caso de existir algún área de las clases de afectación, se puede calcular el área del vehículo y el daño total.
            car_area = search_for_mask(result, img_directory) 
            damage= damage_image(car_area, dent_area, scratch_area) # Optiene el daño de afectación del vehículo.
        else: # En caso de no encontrarse áreas de afectación, se agregarán resultados predeterminados.
            car_area= 0
            damage= 0

        data.append([img_directory, dent_area, scratch_area, car_area, f'{damage}px^2'])
        
    df=pd.DataFrame(data, columns=columnas) # Se crea un DataFrame a partir de los datos.
    return df  

def damage_image(car_area, dent_area, scratch_area):
    if dent_area > 0 and scratch_area > 0: # Comprueba si ambas áreas de la clase de afectación son mayores que 0.
        total_affected_area = abs((dent_area + scratch_area) - car_area) # Calcula el área total afectada restando el área del coche a la suma de las áreas de abolladuras y rayón.
    else:
        total_affected_area = abs(dent_area - car_area) if dent_area > 0 else abs(scratch_area - car_area) if scratch_area > 0 else 0 # Si sólo una de las áreas de afectación es mayor que 0, usa esa área para restarla al área del vehiculo.
    return total_affected_area # Devuelve un float que representa el área total afectada del coche.

df= process_main_detection(results_afectation, results_car)

styled_df = df.style.set_properties(**{'text-align': 'right'})
styled_df


,imagen,area abolladura,area rayon,area carro,afectacion
0,20170531_010133.jpg,0,48466,1230189,1181723px^2
1,5-320w-320w.jpg,25785,0,66753,40968px^2
2,Car-Dent.png,96449,0,150979,54530px^2
3,Charlotte-Toyota-service-1-1024x683.jpg,0,6711,597363,590652px^2
4,GSDH75MG6FAM7FSG43RFQLN5R4.jpg,129872,0,642499,512627px^2
5,OIP (1).jpg,0,9096,0,9096px^2
6,OIP (2).jpeg,43972,0,0,43972px^2
7,OIP (4).jpeg,0,0,0,0px^2
8,OIP(3).jpeg,0,0,0,0px^2
9,R.jpeg,278048,0,0,278048px^2


In [42]:
def calculate_area(vertices): 
    individual_boxes_area= [] # Inicializa una lista vacía para almacenar el área de cada caja delimitadora.
    if isinstance(vertices, list) and not vertices: # Verifica si 'vertices' es una lista vacía.
        return 0 # Si no hay vértices, retorna 0 como área total.
    else: # Si 'vertices' contiene datos:
        for point in vertices: # Itera sobre cada conjunto de coordenadas en 'vertices' que tiene un formato: x [0](coordenada x), y[1](coordenada y), w[2](ancho), h[3](alto) de las cajas delimitadoras. 
            box_area = torch.round(point[2] * point[3]).int() # Calcula el área de la caja (ancho * alto) y lo redondea a un entero, obtiene como resultado un formato: tensor(1230189, dtype=torch.int32).
            individual_boxes_area.append(box_area.item()) # Convierte el tensor 'box_area' a un valor entero y lo añade a la lista 'individual_boxes_area'.
    return individual_boxes_area # Retorna la lista de áreas individuales de las cajas delimitadoras según la clase.

def search_for_mask(results, image_instance):
    resultado= [] # Inicializa una lista vacía para almacenar las áreas de las clases de afectación en la imagen.
    for attribute in results: 
        if os.path.basename(attribute.path) == image_instance: # Verifica que el resultado pertenece a la imagen actual.
            if (attribute.boxes.cls != 2.).all(): # Verifica si los resultados no corresponden a la clase de vehículo (clase 2).
                # Cuando se manejan múltiples clases de objetos (hiperparámetro de predicción classes= []) en una imagen, es necesario separar las coordenadas de cada clase detectada.
                dent_coordinates = [] # Inicializa una lista temporal para las coordenadas de abolladura.
                scratch_coordinates = [] # Inicializa una lista temporal para las coordenadas de rayón.
                tensor = attribute.boxes.cls
                for position, value in enumerate(tensor): # Itera sobre los valores del tensor y sus posiciones.
                    if value == 0.: # Si el valor del tensor es 0, corresponde a una detección de abolladura.
                        dent_coordinates.append(attribute.boxes.xywh[position]) # Añade las coordenadas de la máscara a la lista de abolladuras.
                    elif value == 1.:  # Asegura que solo maneje dos clases de afectación.
                        scratch_coordinates.append(attribute.boxes.xywh[position]) # Añade las coordenadas de la máscara a la lista de rayon.
                dent_area, scratch_area = calculate_area(dent_coordinates), calculate_area(scratch_coordinates) # Calcula el área para cada clase de afectación.
                resultado.append((dent_area, scratch_area)) # Añade los resultados a la lista principal. 
                return resultado 
            else: # Si los resultados corresponden a la clase de vehículo (clase 2), calcula y retorna el área del vehículo.
                return calculate_area(attribute.boxes.xywh) if attribute.boxes is not None else 0
    return 0 # En caso de no haber resultados de vehiculo para la imagen, se retornará 0.

def process_segmentation_results(class_afectation, car): # Función main que controla todo el proceso.
    columnas= ['imagen', 'ClaseMascara', 'area x mascara', 'afectacion x imagen'] # Se definen las columnas del DataFrame.
    data= [] # Se inicializa una lista vacía para almacenar los resultados de cada imagen.

    for attribute in car: # Se recorren principalmente los resultados de carro, ya que es el primer filtro por el que pasa el modelo de segmentación, de esta manera en la salida se podrán ver los resultados para todo el conjunto de datos original.
        img_directory = os.path.basename(attribute.path) # Extrae el nombre base de la imagen de control.
        
        image_data = search_for_mask(class_afectation, img_directory) # Buscar máscaras en la imagen de control.     
        if image_data: 
            for _, index in enumerate(image_data): # Recorre cada resultado de la imagen encontrada para la imagen de control.
                dent_area, scratch_area = index # Extraer la imagen asociada, el área de abolladura y el área de rayón.
                
            if dent_area is not None or scratch_area is not None: # Si existe un area de afectación, se calcula el área del coche y el daño total.
                car_area = search_for_mask(attribute, img_directory) # Recibirá una lista que contendrá los valores de área total para detecciones de vehículos; por ejemplo, [87916, 5504] indica que encontró dos vehículos para la imagen control. 
                damage_image= afectacion_imagen(car_area, dent_area, scratch_area) # Optiene el daño de afectación para cada imagen.
                
            else: # En caso de no encontrarse áreas de afectación, se agregarán resultados predeterminados.
                car_area= 0
                damage_image = 0
            
            # Para añadir los resultados obtenidos se utiliza el formato: [nombre_imagen_control, identificador_clase, areas_individuales, daño_total_imagen]con el fin de buscar en cada imagen los resultados de todas las clases manejadas y evitar errores de la función explode de pandas en las listas.
            data.append([img_directory, 'abolladura', dent_area, damage_image])
            data.append([img_directory, 'rayon', scratch_area, damage_image])
            data.append([img_directory, 'carro', car_area, damage_image])
        
    orden_clases=['abolladura', 'rayon', 'carro'] # Esta lista se creó simplemente con el fin de mantener un orden en la visualización de las clases para el dataframe.
    df= pd.DataFrame(data, columns=columnas) # Se crea un DataFrame a partir de los datos.
    df['Clase'] = pd.Categorical(df['ClaseMascara'], categories=orden_clases, ordered=True)
    grouped_columns=df.groupby(['imagen', 'afectacion x imagen', 'Clase'], group_keys=True, as_index=True, observed=False)[['area x mascara']].apply(lambda x : x) # Agrupa el DataFrame por imagen y aplicar la función lambda.
    grouped_columns_expanded = grouped_columns.explode('area x mascara') # Expande el formato de lista inicial de la columna 'area mascara' en diferentes filas manteniendo su eje en la agrupación.
    return grouped_columns_expanded

def sum_if_list(area):
    return sum(area) if isinstance(area, list) else 0 # Si el parametro es una lista, retornara la suma del contenido de lo contrario devolvera un 0.

def afectacion_imagen(car_area, dent_area, scratch_area): 
    # Como el parametro 'dent_area y scratch area' son listas con formato [área total por objeto según clase], se requiere obtener el resultado total de cada clase. 
    sum_dent = sum_if_list(dent_area) # Recibe la suma total de áreas de abolladuras en una imagen.
    sum_scratch = sum_if_list(scratch_area) # Recibe la suma total de áreas de rayon en una imagen.
    sum_car = sum_if_list(car_area) 
    
    if sum_dent  > 0 and sum_scratch > 0:
        total_affected_area = abs((sum_dent + sum_scratch) - sum_car) # Si ambos tipos de afectacion existen en una imagen, se sumaran primero y restaran con el area del carro.
    else:
        total_affected_area = abs(sum_car - sum_dent) if sum_dent > 0 else abs(sum_car - sum_scratch) if sum_scratch > 0 else 0 # Si sólo una de las áreas es mayor que 0, usa esa área para restarla al área del vehiculo.
    
    return total_affected_area

dataframe = process_segmentation_results(results_afectation, results_car)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
dataframe

area x mascara
imagen                                             afectacion x imagen Clase                       
20170531_010133.jpg                                1181723             abolladura 0               0
                                                                       rayon      1           48466
                                                                       carro      2         1230189
5-320w-320w.jpg                                    40968               abolladura 3           25785
                                                                       rayon      4               0
                                                                       carro      5           66753
Car-Dent.png                                       54530               abolladura 6           96449
                                                                       rayon      7               0
                                                                       carro      8          150979
Charlotte-Toyota-service-1-1024x683.jpg            590652              abolladura 9               0
                                                                       rayon      10           2159
                                                                                  10           4552
                                                                       carro      11         597363
GSDH75MG6FAM7FSG43RFQLN5R4.jpg                     512627              abolladura 12          62473
                                                                                  12          67399
                                                                       rayon      13              0
                                                                       carro      14         642499
OIP (1).jpg                                        9096                abolladura 15              0
                                                                       rayon      16           4406
                                                                                  16           4690
                                                                       carro      17              0
OIP (2).jpeg                                       43972               abolladura 18          43972
                                                                       rayon      19              0
                                                                       carro      20              0
OIP (4).jpeg                                       0                   abolladura 21              0
                                                                       rayon      22              0
                                                                       carro      23         762729
OIP(3).jpeg                                        0                   abolladura 24              0
                                                                       rayon      25              0
                                                                       carro      26         144130
R.jpeg                                             278048              abolladura 27         278048
                                                                       rayon      28              0
                                                                       carro      29              0
R.jpg                                              73506               abolladura 30         145368
                                                                                  30         165880
                                                                       rayon      31              0
                                                                       carro      32         384754
R2.jpeg                                            86780               abolladura 33          17211
                                                                       rayon      34 